Graf yükleniyor...
Düğüm sayısı: 123

GCN modeli çalıştırılıyor...
Epoch 50/200, Loss: 1.3471
Epoch 100/200, Loss: 1.3694
Epoch 150/200, Loss: 1.3228
Epoch 200/200, Loss: 1.3393
En iyi küme sayısı aranıyor (3-20 arası)...
Seçilen küme sayısı: 20 (Score: 2332.4651)

GCN_GNN Tamamlandı (52.77 sn)
Toplam Küme Sayısı: 20
Embeddingler kaydedildi: ./ValidationGeo/community_results\gcn_gnn_embeddings.csv

GAT modeli çalıştırılıyor...
Epoch 50/200, Loss: 22.8444
Epoch 100/200, Loss: 19.7017
Epoch 150/200, Loss: 20.0336
Epoch 200/200, Loss: 25.5380
En iyi küme sayısı aranıyor (3-20 arası)...
Seçilen küme sayısı: 20 (Score: 2891.1374)

GAT_GNN Tamamlandı (56.34 sn)
Toplam Küme Sayısı: 20
Embeddingler kaydedildi: ./ValidationGeo/community_results\gat_gnn_embeddings.csv

İşlem tamamlandı.
Düğüm sayısı: 123

GCN modeli çalıştırılıyor...
Epoch 50/200, Loss: 1.3846
Epoch 100/200, Loss: 1.3601
Epoch 150/200, Loss: 1.3571
Epoch 200/200, Loss: 1.3597
En iyi küme sayısı aranıyor (3-20 arası)...
Seçilen k

C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
Traceback (most recent call last):
  File "C:\Users\kahra\AppData\Local\Temp\ipykernel_36044\2548695269.py", line 19, in <module>
    communities, embeddings, elapsed_time, node_list = run_gnn_community_detection(
  File "C:\Users\kahra\AppData\Local\Temp\ipykernel_36044\4111405331.py", line 237, in run_gnn_community_detection
    communities = cluster_embeddings(embeddings)
  File "C:\Users\kahra\AppData\Local\Temp\ipykernel_36044\4111405331.py", line 124, in cluster_embeddings
    score = calinski_harabasz_score(embeddings_np, labels)
  File "C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "C:\Users

Epoch 50/200, Loss: 1.3715
Epoch 100/200, Loss: 1.3603
Epoch 150/200, Loss: 1.3772
Epoch 200/200, Loss: 1.3789
En iyi küme sayısı aranıyor (3-20 arası)...
Seçilen küme sayısı: 20 (Score: 17596.0009)

GCN_GNN Tamamlandı (22.23 sn)
Toplam Küme Sayısı: 20
Embeddingler kaydedildi: ./TrainingWDE/community_results\gcn_gnn_embeddings.csv

GAT modeli çalıştırılıyor...
Epoch 50/200, Loss: 21.2440
Epoch 100/200, Loss: 25.2966
Epoch 150/200, Loss: 25.0486
Epoch 200/200, Loss: 18.7076
En iyi küme sayısı aranıyor (3-20 arası)...
Seçilen küme sayısı: 20 (Score: 8414.4155)

GAT_GNN Tamamlandı (26.46 sn)
Toplam Küme Sayısı: 20
Embeddingler kaydedildi: ./TrainingWDE/community_results\gat_gnn_embeddings.csv

İşlem tamamlandı.


In [2]:
#!/usr/bin/env python
# coding: utf-8

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
import numpy as np
import time
import csv
import os
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score, silhouette_score
from sklearn.preprocessing import StandardScaler

# --- GCN Model Tanımı ---
class GCN_Community(torch.nn.Module):
    def __init__(self, num_features, hidden_dim=64, embedding_dim=32):
        super(GCN_Community, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, embedding_dim)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # 1. Katman
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # 2. Katman
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # 3. Katman (Embedding Çıktısı)
        x = self.conv3(x, edge_index)
        return x

# --- GAT Model Tanımı ---
class GAT_Community(torch.nn.Module):
    def __init__(self, num_features, hidden_dim=64, embedding_dim=32, heads=4):
        super(GAT_Community, self).__init__()
        self.conv1 = GATConv(num_features, hidden_dim, heads=heads, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * heads, embedding_dim, heads=1, concat=False, dropout=0.6)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# --- Negatif Örnekleme (Loss fonksiyonu için) ---
def negative_sampling(data, num_neg_samples=None):
    num_nodes = data.num_nodes
    num_edges = data.edge_index.size(1)
    if num_neg_samples is None:
        num_neg_samples = num_edges
    
    neg_edges = []
    edge_set = set(map(tuple, data.edge_index.t().tolist()))
    
    while len(neg_edges) < num_neg_samples:
        i = np.random.randint(0, num_nodes)
        j = np.random.randint(0, num_nodes)
        if i != j and (i, j) not in edge_set and (j, i) not in edge_set:
            neg_edges.append([i, j])
    
    return torch.tensor(neg_edges, dtype=torch.long).t().to(data.edge_index.device)

# --- Eğitim Fonksiyonu ---
def train_gnn(model, data, optimizer, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        embeddings = model(data)
        
        # Reconstruction Loss
        pos_edge_index = data.edge_index
        neg_edge_index = negative_sampling(data)
        
        # Pozitif kenarların skoru (birbirine yakın olmalı)
        pos_loss = -torch.log(
            torch.sigmoid(
                (embeddings[pos_edge_index[0]] * embeddings[pos_edge_index[1]]).sum(dim=1)
            ) + 1e-15
        ).mean()
        
        # Negatif kenarların skoru (birbirine uzak olmalı)
        neg_loss = -torch.log(
            1 - torch.sigmoid(
                (embeddings[neg_edge_index[0]] * embeddings[neg_edge_index[1]]).sum(dim=1)
            ) + 1e-15
        ).mean()
        
        loss = pos_loss + neg_loss
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
    return model

# --- Geliştirilmiş Node Features ---
def create_rich_node_features(G_nx):
    """
    Daha zengin node feature'ları oluşturur:
    - Degree
    - Clustering coefficient
    - PageRank
    - Betweenness centrality (sadece küçük graflarda)
    """
    node_list = list(G_nx.nodes())
    features = []
    
    # Degree
    degrees = dict(G_nx.degree())
    
    # Clustering coefficient
    clustering = nx.clustering(G_nx)
    
    # PageRank
    pagerank = nx.pagerank(G_nx, max_iter=100)
    
    # Betweenness (sadece küçük graflarda hesapla)
    if G_nx.number_of_nodes() < 500:
        betweenness = nx.betweenness_centrality(G_nx)
    else:
        betweenness = {node: 0.0 for node in node_list}
    
    for node in node_list:
        features.append([
            degrees[node],
            clustering[node],
            pagerank[node],
            betweenness[node]
        ])
    
    features_array = np.array(features, dtype=np.float32)
    
    # Normalize et
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features_array)
    
    return torch.tensor(features_normalized, dtype=torch.float)

# --- İyileştirilmiş Clustering ---
def cluster_embeddings(embeddings, min_clusters=2, max_clusters=20):
    """
    Geliştirilmiş kümeleme fonksiyonu:
    - Embedding diversity kontrolü
    - Silhouette score ile yedekleme
    - Daha güvenli hata yönetimi
    """
    # Tensor kontrolü
    if torch.is_tensor(embeddings):
        embeddings_np = embeddings.detach().cpu().numpy()
    else:
        embeddings_np = embeddings
    
    n_samples = len(embeddings_np)
    
    # Embedding çeşitliliğini kontrol et
    unique_embeddings = np.unique(embeddings_np, axis=0)
    print(f"Benzersiz embedding sayısı: {len(unique_embeddings)}/{n_samples}")
    
    if len(unique_embeddings) < min_clusters:
        print(f"⚠️ UYARI: Çok az benzersiz embedding ({len(unique_embeddings)}). Tüm noktalar tek kümede.")
        # Fallback: Tüm noktaları tek kümeye ata
        return {0: list(range(n_samples))}
    
    # Küme sayısı limitlerini ayarla
    max_possible_clusters = min(max_clusters, len(unique_embeddings) - 1, n_samples - 1)
    min_clusters = max(2, min_clusters)
    
    if max_possible_clusters < min_clusters:
        print(f"⚠️ UYARI: Yeterli veri yok. Tüm noktalar tek kümede.")
        return {0: list(range(n_samples))}
    
    # Normalize embeddings
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings_np)
    
    best_score = -1
    best_k = min_clusters
    best_method = None
    
    print(f"En iyi küme sayısı aranıyor ({min_clusters}-{max_possible_clusters} arası)...")
    
    for k in range(min_clusters, max_possible_clusters + 1):
        try:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
            labels = kmeans.fit_predict(embeddings_scaled)
            
            # Gerçekten k farklı küme var mı kontrol et
            n_unique_labels = len(np.unique(labels))
            
            if n_unique_labels < 2:
                continue
            
            # Calinski-Harabasz skorunu dene
            try:
                ch_score = calinski_harabasz_score(embeddings_scaled, labels)
                
                # Silhouette score ile doğrula
                sil_score = silhouette_score(embeddings_scaled, labels)
                
                # Kombine skor (CH skoruna öncelik ver)
                combined_score = ch_score * 0.7 + (sil_score + 1) * 1000 * 0.3
                
                if combined_score > best_score:
                    best_score = combined_score
                    best_k = k
                    best_method = "CH+Silhouette"
                    
            except ValueError as e:
                # CH skoru hesaplanamadıysa, sadece silhouette kullan
                if n_unique_labels >= 2:
                    sil_score = silhouette_score(embeddings_scaled, labels)
                    if sil_score > best_score:
                        best_score = sil_score
                        best_k = k
                        best_method = "Silhouette"
                        
        except Exception as e:
            print(f"k={k} için hata: {str(e)}")
            continue
    
    print(f"Seçilen küme sayısı: {best_k} (Method: {best_method}, Score: {best_score:.4f})")
    
    # Final kümeleme
    kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(embeddings_scaled)
    
    communities = {}
    for label in np.unique(labels):
        communities[int(label)] = np.where(labels == label)[0].tolist()
    
    return communities

# --- Embeddingleri Kaydetme Fonksiyonu (Path root'a kaydeder) ---
def save_embeddings_to_csv(embeddings, node_list, algorithm_name, path):
    """
    Embeddingleri path klasörünün içine kaydeder.
    Format: NodeID, Emb_0, Emb_1, ..., Emb_N
    """
    filename = os.path.join(path, f"{algorithm_name.lower()}_embeddings.csv")
    
    # Tensor kontrolü
    if torch.is_tensor(embeddings):
        embeddings_np = embeddings.detach().cpu().numpy()
    else:
        embeddings_np = embeddings

    try:
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            
            # Header yaz (NodeID, dim0, dim1...)
            header = ["NodeID"] + [f"dim_{i}" for i in range(embeddings_np.shape[1])]
            writer.writerow(header)
            
            # Her satırı yaz
            for i, node_id in enumerate(node_list):
                row = [node_id] + list(embeddings_np[i])
                writer.writerow(row)
                
        print(f"Embeddingler kaydedildi: {filename}")
        return True
    except Exception as e:
        print(f"Embedding kaydetme hatası: {str(e)}")
        return False

# --- Communities Kaydetme Fonksiyonu (community_results klasörüne kaydeder) ---
def save_communities_to_csv(communities, node_list, algorithm_name, path):
    """
    Kümeleri path/community_results klasörüne kaydeder.
    """
    if not communities: 
        return False
    
    output_dir = os.path.join(path, "community_results")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    filename = os.path.join(output_dir, f"{algorithm_name.lower()}_communities.csv")
    
    try:
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for comm_id, member_indices in communities.items():
                # İndeksleri gerçek düğüm isimlerine çevir
                real_members = [node_list[idx] for idx in member_indices]
                writer.writerow(real_members)
        print(f"Kümeler kaydedildi: {filename}")
        return True
    except Exception as e:
        print(f"Community CSV hatası: {str(e)}")
        return False

def run_gnn_community_detection(G_nx, model_type="gcn", epochs=200, embedding_dim=32):
    print(f"\n{model_type.upper()} modeli çalıştırılıyor...")
    start_time = time.time()
    
    # Düğüm sırasını sabitle (NetworkX node order)
    node_list = list(G_nx.nodes())
    
    data = from_networkx(G_nx)
    
    # Zengin node features oluştur
    print("Node feature'ları oluşturuluyor...")
    data.x = create_rich_node_features(G_nx)
    
    num_features = data.x.size(1)
    print(f"Feature boyutu: {num_features}")
    
    if model_type.lower() == "gcn":
        model = GCN_Community(num_features, hidden_dim=64, embedding_dim=embedding_dim)
    elif model_type.lower() == "gat":
        model = GAT_Community(num_features, hidden_dim=64, embedding_dim=embedding_dim)
    else:
        print(f"Bilinmeyen model: {model_type}")
        return None, None, None, None
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model = train_gnn(model, data, optimizer, epochs=epochs)
    
    # Embeddingleri al
    model.eval()
    with torch.no_grad():
        embeddings = model(data)
    
    # Embedding istatistikleri
    emb_np = embeddings.detach().cpu().numpy()
    print(f"Embedding istatistikleri - Mean: {emb_np.mean():.4f}, Std: {emb_np.std():.4f}")
    
    # Kümele
    communities = cluster_embeddings(embeddings, min_clusters=2, max_clusters=20)
    
    elapsed_time = time.time() - start_time
    
    return communities, embeddings, elapsed_time, node_list


# --- Ana Çalışma Döngüsü ---
paths = ["./ValidationGeo", "./ValidationWDE", "./TrainingGeo", "./TrainingWDE"]
print("Graf yükleniyor...")

for path in paths:
    print(f"\n{'='*60}")
    print(f"İşleniyor: {path}")
    print('='*60)
    
    try:
        # Dosya adını kontrol et
        adjlist_file = os.path.join(path, "my.adjlist")
        if not os.path.exists(adjlist_file):
            print(f"HATA: '{adjlist_file}' dosyası bulunamadı!")
            continue
    
        G_nx = nx.read_adjlist(adjlist_file)
        if nx.is_directed(G_nx):
            G_nx = G_nx.to_undirected()
            
        print(f"Düğüm sayısı: {G_nx.number_of_nodes()}")
        print(f"Kenar sayısı: {G_nx.number_of_edges()}")
        
        gnn_models = ["gcn", "gat"]
        
        for model_type in gnn_models:
            communities, embeddings, elapsed_time, node_list = run_gnn_community_detection(
                G_nx, 
                model_type=model_type,
                epochs=200,
                embedding_dim=32
            )
            
            if communities:
                algo_name = f"{model_type.upper()}_GNN"
                
                print(f"\n{algo_name} Tamamlandı ({elapsed_time:.2f} sn)")
                print(f"Toplam Küme Sayısı: {len(communities)}")
                
                # Küme büyüklüklerini göster
                sizes = [len(members) for members in communities.values()]
                print(f"Küme büyüklükleri: Min={min(sizes)}, Max={max(sizes)}, Ortalama={np.mean(sizes):.1f}")
                
                # 1. Embeddingleri path root'a kaydet
                save_embeddings_to_csv(embeddings, node_list, algo_name, path)
                
                # 2. Kümeleri path/community_results içine kaydet
                save_communities_to_csv(communities, node_list, algo_name, path)
                
        print(f"\n{path} için işlem tamamlandı.")
        
    except Exception as e:
        print(f"Beklenmedik bir hata: {str(e)}")
        import traceback
        traceback.print_exc()

print("\n" + "="*60)
print("TÜM İŞLEMLER TAMAMLANDI")
print("="*60)

Graf yükleniyor...

İşleniyor: ./ValidationGeo
Düğüm sayısı: 123
Kenar sayısı: 3694

GCN modeli çalıştırılıyor...
Node feature'ları oluşturuluyor...
Feature boyutu: 4
Epoch 50/200, Loss: 0.9680
Epoch 100/200, Loss: 0.9234
Epoch 150/200, Loss: 0.8570
Epoch 200/200, Loss: 0.8674
Embedding istatistikleri - Mean: 0.0147, Std: 0.2670
Benzersiz embedding sayısı: 121/123
En iyi küme sayısı aranıyor (2-20 arası)...
Seçilen küme sayısı: 20 (Method: CH+Silhouette, Score: 775.8253)

GCN_GNN Tamamlandı (50.64 sn)
Toplam Küme Sayısı: 20
Küme büyüklükleri: Min=2, Max=15, Ortalama=6.2
Embeddingler kaydedildi: ./ValidationGeo\gcn_gnn_embeddings.csv
Kümeler kaydedildi: ./ValidationGeo\community_results\gcn_gnn_communities.csv

GAT modeli çalıştırılıyor...
Node feature'ları oluşturuluyor...
Feature boyutu: 4
Epoch 50/200, Loss: 1.3164
Epoch 100/200, Loss: 1.2058
Epoch 150/200, Loss: 1.3498
Epoch 200/200, Loss: 1.2858
Embedding istatistikleri - Mean: -0.0179, Std: 0.1435
Benzersiz embedding sayısı: 122/1